# DOTA2 предсказание победителя

In [48]:
import pandas as pd
features = pd.read_csv('./features.csv', index_col='match_id')
features_test = pd.read_csv('./features_test.csv', index_col='match_id')
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [49]:
#Определяем количество уникальных героев
import numpy as np
l=[]
k=['r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero']
for i in k:
    l.append(features[i].unique()) 
    l.append(features_test[i].unique())
N=np.max(l) #так как в индентификаторов героев есть пропуски, то берем максимальный идентификатор героя, чтобы учесть пропуски

#### Описание признаков в таблице

- `match_id`: идентификатор матча в наборе данных
- `start_time`: время начала матча (unixtime)
- `lobby_type`: тип комнаты, в которой собираются игроки (расшифровка в `dictionaries/lobbies.csv`)
- Наборы признаков для каждого игрока (игроки команды Radiant — префикс `rN`, Dire — `dN`):
    - `r1_hero`: герой игрока (расшифровка в dictionaries/heroes.csv)
    - `r1_level`: максимальный достигнутый уровень героя (за первые 5 игровых минут)
    - `r1_xp`: максимальный полученный опыт
    - `r1_gold`: достигнутая ценность героя
    - `r1_lh`: число убитых юнитов
    - `r1_kills`: число убитых игроков
    - `r1_deaths`: число смертей героя
    - `r1_items`: число купленных предметов
- Признаки события "первая кровь" (first blood). Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение
    - `first_blood_time`: игровое время первой крови
    - `first_blood_team`: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)
    - `first_blood_player1`: игрок, причастный к событию
    - `first_blood_player2`: второй игрок, причастный к событию
- Признаки для каждой команды (префиксы `radiant_` и `dire_`)
    - `radiant_bottle_time`: время первого приобретения командой предмета "bottle"
    - `radiant_courier_time`: время приобретения предмета "courier" 
    - `radiant_flying_courier_time`: время приобретения предмета "flying_courier" 
    - `radiant_tpscroll_count`: число предметов "tpscroll" за первые 5 минут
    - `radiant_boots_count`: число предметов "boots"
    - `radiant_ward_observer_count`: число предметов "ward_observer"
    - `radiant_ward_sentry_count`: число предметов "ward_sentry"
    - `radiant_first_ward_time`: время установки командой первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля
- Итог матча (данные поля отсутствуют в тестовой выборке, поскольку содержат информацию, выходящую за пределы первых 5 минут матча)
    - `duration`: длительность
    - `radiant_win`: 1, если победила команда Radiant, 0 — иначе
    - Состояние башен и барраков к концу матча (см. описание полей набора данных)
        - `tower_status_radiant`
        - `tower_status_dire`
        - `barracks_status_radiant`
        - `barracks_status_dire`

## Градиентный бустинг

In [50]:
import numpy as np
y_train=features['radiant_win'] #целевая переменная
#Удаляем колонки связанные с итогом матчей
features.drop(['duration','radiant_win',
               'tower_status_radiant',
               'tower_status_dire',
               'barracks_status_radiant',
               'barracks_status_dire'], axis=1, inplace=True)

In [51]:
#Смотрим на пропуски
z=[]
for i in range(0,len(features.count())):
    if features.count()[i]<97230:
        z.append(i)
#Признаки в которых есть пропуски        
for j in z:
    print(features.columns[j])
    
#Заменяем пропущенные значения 0
features.fillna(0,inplace=True)
features_test.fillna(0,inplace=True)

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


Почему значения признаков могут быть пропущены? Ну например для первых четырех признаков все достаточно логично, это признаки "первой крови" и как уже указано в описании первая кровь может не произойти в первые 5 минут, а значит эти признаки принимают пропущенное значение. Остальные признаки характеризующие каждую команду, например приобретения курьера, также может не произойти в первые 5 минут, поэтому в нем имеются пропуски.

In [138]:

#Непосредственно начинаем градиентный бустинг
import time
import datetime
start_time = datetime.datetime.now()
from sklearn.ensemble import GradientBoostingClassifier as gbc
from sklearn.ensemble import GradientBoostingRegressor as gbr
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kf=KFold(n_splits=5, shuffle=True)
n=30
clf = gbc(n_estimators=n,verbose=True)
clf.fit(features, y_train)
predictions = clf.predict_proba(features_test)[:, 1]
quality=cross_val_score(clf, features, y_train, cv=kf, scoring='roc_auc')
print('Кол-во деревьев =',n,"качество",np.mean(quality))
print('Time elapsed:', datetime.datetime.now() - start_time)

      Iter       Train Loss   Remaining Time 
         1           1.3786           14.20s
         2           1.3732           14.00s
         3           1.3681           13.16s
         4           1.3636           12.56s
         5           1.3589           11.72s
         6           1.3547           11.35s
         7           1.3502           10.80s
         8           1.3461           10.19s
         9           1.3422            9.60s
        10           1.3385            9.08s
        20           1.3092            4.48s
        30           1.2892            0.00s
      Iter       Train Loss   Remaining Time 
         1           1.3784           10.72s
         2           1.3728           10.25s
         3           1.3678            9.84s
         4           1.3634            9.46s
         5           1.3589            9.12s
         6           1.3543            8.62s
         7           1.3499            8.16s
         8           1.3457            7.74s
        

Выводы:
2. Столбец содержащий целевую переменную  radiant_win
3. Кросс-валидация проводилась  0:01:08.559378, так как параметр random_state не установлен, время может незначительно отклоняться. Качество с точностью до 4 знаков составило 0.6895
4. В данном случае прирост качестве при увеличении кол-ва деревьев не столь значительный, например для 50 деревьев качество составило 0.6977, поэтому имеет смысл остановиться на 30 деревьях. Для ускорения обучения можно снизить кол-во блоков на которые разбивается выборка при кросс-валидации или уменьшить глубину деревьев в градиентном бустинге.

## Логистическая регрессия

In [149]:
import numpy as np
from sklearn.linear_model import LogisticRegression as lr
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import scale
import time
import datetime
start_time = datetime.datetime.now()
features_s=scale(features)
features_s_test=scale(features_test)
kf=KFold(n_splits=5, shuffle=True)
clf = lr(penalty='l2',C=1)
clf1=lr(penalty='l2',C=10)
clf.fit(features_s, y_train)
predictions = clf.predict_proba(features_s_test)[:, 1]
quality=cross_val_score(clf, features_s, y_train, cv=kf, scoring='roc_auc')
quality1=cross_val_score(clf1, features_s, y_train, cv=kf, scoring='roc_auc')
print(np.mean(quality))
print(np.mean(quality1))
print('Time elapsed:', datetime.datetime.now() - start_time)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

0.7164742891835044
0.7164443996137732
Time elapsed: 0:00:25.343089


In [204]:
#Удаляем категориальные признаки
features.drop(['lobby_type','r1_hero','r2_hero',
               'r3_hero',
               'r4_hero',
               'r5_hero',
               'd1_hero',
               'd2_hero',
               'd3_hero',
               'd4_hero',
               'd5_hero'], axis=1)
features_test.drop(['lobby_type','r1_hero','r2_hero',
               'r3_hero',
               'r4_hero',
               'r5_hero',
               'd1_hero',
               'd2_hero',
               'd3_hero',
               'd4_hero',
               'd5_hero'], axis=1)
#Таже лог регрессия, но с удалением категориальных переменных
start_time = datetime.datetime.now()
features_s=scale(features)
features_s_test=scale(features_test)
kf=KFold(n_splits=5, shuffle=True)
clf = lr(penalty='l2',C=1)
clf1=lr(penalty='l2',C=0.01) #сравнение с различными парметрами регуляризации показало, что оптимальный С=0.01
clf.fit(features_s, y_train)
predictions = clf.predict_proba(features_s_test)[:, 1]
quality=cross_val_score(clf, features_s, y_train, cv=kf, scoring='roc_auc')
quality1=cross_val_score(clf1, features_s, y_train, cv=kf, scoring='roc_auc')
print(np.mean(quality))
print(np.mean(quality1))
print('Time elapsed:', datetime.datetime.now() - start_time)

In [52]:
import numpy as np
from sklearn.linear_model import LogisticRegression as lr
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import scale
import time
import datetime
# N — количество различных героев в выборке
X_pick = np.zeros((features.shape[0], N))
X_pick_test=np.zeros((features_test.shape[0], N))
for i, match_id in enumerate(features.index):
    for p in range(5):
        X_pick[i, features.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
xpick_df = pd.DataFrame(X_pick)
xpick_df = xpick_df.astype(int)
xpick_df.index = features.index
features = features.join(xpick_df, how="inner")
#Для тестовой выборки
for i, match_id in enumerate(features_test.index):
    for p in range(5):
        X_pick_test[i, features_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_test[i, features_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
xpick_df_test = pd.DataFrame(X_pick_test)
xpick_df_test = xpick_df_test.astype(int)
xpick_df_test.index = features_test.index
features_test = features_test.join(xpick_df_test, how="inner") 
#Удаляем категориальные признаки и оставляем замененные
features.drop(['lobby_type','r1_hero','r2_hero',
               'r3_hero',
               'r4_hero',
               'r5_hero',
               'd1_hero',
               'd2_hero',
               'd3_hero',
               'd4_hero',
               'd5_hero'], axis=1)
features_test.drop(['lobby_type','r1_hero','r2_hero',
               'r3_hero',
               'r4_hero',
               'r5_hero',
               'd1_hero',
               'd2_hero',
               'd3_hero',
               'd4_hero',
               'd5_hero'], axis=1)
#Таже лог регрессия, но с удалением категориальных переменных
start_time = datetime.datetime.now()
features_s=scale(features)
features_s_test=scale(features_test)
kf=KFold(n_splits=5, shuffle=True)
clf = lr(penalty='l2',C=1)
clf1=lr(penalty='l2',C=0.01)
clf.fit(features_s, y_train)
predictions = clf.predict_proba(features_s_test)[:, 1]
quality=cross_val_score(clf, features_s, y_train, cv=kf, scoring='roc_auc')
quality1=cross_val_score(clf1, features_s, y_train, cv=kf, scoring='roc_auc')
print(np.mean(quality))
print(np.mean(quality1)) #сравнение с различными парметрами регуляризации показало, что оптимальный С=0.01
print(min(predictions))
print(max(predictions))
print('Time elapsed:', datetime.datetime.now() - start_time)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarni

0.7517843300185986
0.7518615519492222
Time elapsed: 0:00:44.765965


Выводы:
1. Качество логистической регрессии над всеми признаками получилось 0.7164. Качество оказалось больше, чем у градиентного бустинга. Благодаря тому, как обучается модель, предсказания логистической регрессии можно использовать для отображения вероятности принадлежности образца к классу 0 или 1, наши данные как раз хорошо подходят под эту классификацию и возможно имеют линейную или около линейнуб природу. Логистическая регрессия работает быстрее, чем градиентный бустинг.
2. Новое значение метрики качества 0.7165. Незначительное изменения можно объяснить тем, что мы убрали похожие возможно коррелирующие признаки, которые мешают в оценке регрессии.
3. В данной выборке 108 различных идентификаторов героев.
4. Качество при добавлении мешка слов составило 0.7517. По сравнению с пунктом 2 качество улучшилось, это можно объяснить тем, что мы сумели классифицировать объекты и использовать их как бинарные переменные, чтобы учесть значимость влияния героев на победу команды.
5. Минимальное значение 0.0089  , максимальное значение 0.9965 с округлением до 4 знаков.